## Загрузка датасета на HuggingFace

Этот ноутбук дает пример того, как залить локальный датасет на ХФ. Адаптируйте его под свой датасет. Затем, выложите в гитхаб получившийся ноутбук (приложите к своему датасету), чтобы всегда был доступен код для заливки вашего датасета на ХФ. Убедитесь, что ячейки последовательно запускаются.

In [ ]:
import json
import datasets
from tqdm import tqdm
import os

In [ ]:
datasets.Sequence()

Value(dtype='string', id=None)

### Подготовка данных

#### WARNING! 

Если ваш датасет является __ПРИВАТНЫМ__, то оставьте `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS` равным `True`. Иначе, поставьте `False`. Этот флаг дальше используется, чтобы стереть ответы перед загрузкой на ХФ датасета. На ХФ даже приватно не должно лежать датасетов с ответами!

In [ ]:
MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS = False

Данный пример рассчитан на загрузку на ХФ локального датасета.

Параметр `path_to_data` - это путь ДО файлов `shots.json` и `test.json`, которые вы будете дальше загружать на ХФ в виде датасета или домена датасета. 

Параметр `path_to_meta` - это путь ДО меты датасета.

Итоговые пути будут собираться из `path_to_data` / `path_to_meta` + `file_name.json`!

In [ ]:
path_to_data = "./"
path_to_meta = "./"

Сплиты и мета лежат в формате JSON.

In [ ]:
def load_json(path):
    with open(path) as f:
        data = json.load(f)
    return data

#### Подгрузка данных

Считайте сплиты и мету датасета (домена датасета). Это просто JSON файлики либо внутри прямо папки датасета, либо внутри папки по названию домена, который вы будете загружать.

In [ ]:
shots = load_json(os.path.join(path_to_data, "shots.json"))["data"]
test = load_json(os.path.join(path_to_data, "test.json"))["data"]
meta = load_json(os.path.join(path_to_meta, "dataset_meta.json"))

Из меты для датасета нужны только промпты.

In [ ]:
prompts = meta["prompts"]

#### Обработка полей датасета

На ХФ вы загружаете датасет, где у КАЖДОГО сэмпла вместо числа в поле instruction стоит промпт. Число указывает, какой по индексу взять промпт из секции с промптами в мете датасета.

In [ ]:
for card in shots:
    card["instruction"] = prompts[card["instruction"]]

for card in test:
    card["instruction"] = prompts[card["instruction"]]

#### Убираем ответы для приватных задач

Надеемся, вы поставили в начале ноутбука корректное значение `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS`.

Если там стоит `True`, то в `test` сплите ответы на все задания стираются. Вместо них остается пустая строка, чтобы вы случайно не пушнули на ХФ датасет с заполненными ответами, и они не утекли.

In [ ]:
def hide_answers(dataset_split: list[dict]):
    for card in tqdm(dataset_split):
        card["outputs"] = ""

In [ ]:
if MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS:
    hide_answers(test)

### Создаем датасет для загрузки на ХФ

#### Аннотация полей датасета

В `features` повторяется структура КАЖДОГО сэмпла вашего датасета с описанием формата данных в каждом поле. 
- `instruction` всегда строка
- `meta` - `id` всегда целое число

Далее смотрите по тому, какие поля у вашего датасета.

`features` нужен для того, чтобы ХФ сам автоматически создал техническую часть README.md датасета, заполнив ее информацией, которая используется при загрузке датасета. Отсутствие `features` может и обычно приводит к невозможности использовать датасет. Ровно такие же последствия будут от ошибок в заполнении (например, неправильно указан тип данных).

__Внимание!__ Если у вас в датасете в разных вопросах разное количество ответов, то поля в `features` нужно заполнить для сэмпла с НАИБОЛЬШИМ количеством ответов. Иначе говоря, представьте, что у вас у всех вопросов в датасете максимальное количество вариантов ответа, просто некоторые пустые. Вот из такого соображения и заполняйте `features`. Он один на весь датасет и должен охватывать все поля, которые в нем встречаются!

In [4]:
features = datasets.Features({
    "instruction": datasets.Value("string"),
    "inputs": {
        "function": datasets.Value("string"),
        "tests": datasets.Value("string")
        # other features
    },
    "outputs": datasets.Value("string"),
    "meta": {
        "id": datasets.Value("int32"),
        "canonical_solution": datasets.Value("string"),
        "entry_point": datasets.Value("string")
        # other features
    },
})


#### Создание датасетов для каждого сплита

Теперь создаем сплиты датасета. Можно это сделать либо в одну строку:

In [ ]:
shots_ds = datasets.Dataset.from_list(shots, features=features)

Но это способ для маленьких датасетов. Большие датасеты так создаются крайне долго. Чтобы побыстрее собрать большой датасет, можно разбить его на кусочки по N сэмплов. Перегонять каждый кусочек и присоединять к уже конвертированным ранее кусочкам.

In [ ]:
STEP = 20

lst_steps = []
for i in tqdm(range(0, len(test), STEP)):
    tmp = datasets.Dataset.from_list(test[i: i+STEP], features=features)
    lst_steps.extend([tmp])
    
test_ds = datasets.concatenate_datasets(lst_steps)

##### Проверка

Если вы собирали датасет по кускам, то разумно будет проверить, что сборка прошла успешно - ничего не потеряно, не продублировано и так далее.

Но вы можете проверить целостность датасета даже, если и не по кусочкам собирали его. Так вы можете отловить ошибки до того, как их найдут на ревью :)

In [ ]:
# проверка, что id вопросов сходятся

bools = []
for i in range(len(test)):
    bools.extend([test[i]["meta"]["id"] == test_ds[i]["meta"]["id"]])
all(bools)

In [ ]:
# проверка, что количество вопросов до конвертации и после осталось одинаковым

len(test) == len(test_ds)

#### Собираем сплиты в один датасет

In [ ]:
dataset = datasets.DatasetDict({"shots": shots_ds, "test": test_ds})

### Загрузка датасета на ХФ

Для загрузки на ХФ вам понадобятся:
- Токен. Это строка, содержащая ключик, который позволит вам записывать в репозиторий. 
- Путь для записи. Это тоже строка, которая содержит путь, по которому вы выложите свой датасет. Этот путь содержит название аккаунта (MERA-evaluation) и название вашего датасета. Название датасета пишите ровно так, как оно заявлено в мете! Регистр тоже имеет значение!

Советуем опубликовывать сперва всё приватно, и выслать админам MERA токен и путь для верификации. 
Если ваш сет публичный и вы хотите отправить всё публично, то в Merge request просто пришлите путь к сету.

In [ ]:
### TOKEN
token = "token"
###

### UPLOAD PATH
dataset_path_hub = "MERA-evaluation/ruCLEVR"
###


# Если вы хотите предварительно протестировать, как датасет будет выглядеть после заливки на ХФ,
# то можно загрузить его сначала к себе в приватный репозиторий

### UPLOAD PATH
# dataset_path_hub = "artemorloff/ruclevr"
###

In [ ]:
dataset.push_to_hub(dataset_path_hub, private=True, token=token) # опубликовать приватно

### Проверка того, как датасет загрузился на ХФ

После загрузки датасета будет полезно посмотреть, как его будет видеть любой человек, который после вашей загрузки его скачает. 

Загрузите датасет целиком, используя `datasets.load_dataset(dataset_path_hub)`, а затем проверьте, что:
- все поля на месте. Если у вас в датасете у разных вопросов было разное количество вариантов ответа, то теперь их везде станет одинаковое количество. Недостающие варианты ответа у каждого вопроса теперь будут прописаны, но будут иметь значение `None`. Это нормально.
- датасет идентичен по содержанию исходному. То есть, в исходном JSON и загруженном датасете вопрос с одинаковым `id` имеет одинаково заполненные поля (кроме тех, что заполняются `None`, как описано выше).

In [ ]:
ds = datasets.load_dataset(dataset_path_hub)

In [ ]:
ds

Пример проверки двух сплитов, что в них тексты вопросов совпадают с оригинальными

In [ ]:
check = []
for idx, card in enumerate(ds["shots"]):
    same_question = shots[idx]["inputs"]["function"] == card["inputs"]["function"]
    check.extend([same_question])

all(check)

In [ ]:
check = []
for idx, card in enumerate(ds["test"]):
    same_question = test[idx]["inputs"]["function"] == card["inputs"]["function"]
    check.extend([same_question])

all(check)